In [53]:
import pandas as pd
import numpy as np
"""
Author:ZhuJinqiu
Date:2021-4-23
Version:1.0
"""

# 1数据预处理

In [ ]:
# import pandas as pd

def strip(text):
    """
    去掉数据中的空格
    """
    try:
        return text.strip()
    except AttributeError:
        return text

data = pd.read_table(r"苏果便利店销售数据1.csv", sep=r',',
                      #names=["收银员名称", "单据号", "销售时间", "商品货号","商品名称","销售单价",],
                      converters = {'商品名称' : strip,
                                    })

In [56]:
len(data['商品名称'].unique())#商品名称的去重

2028

In [57]:
data = data.drop(data[data['商品名称'] == '合计分舍去'].index)#删除合计行
data = data.drop(data[data['商品名称'] == '华润苏果中号购物袋'].index)#删除购物袋
data = data.drop(data[data['商品名称'] == '华润苏果大号购物袋'].index)
data = data.drop(data[data['商品名称'] == '华润苏果小号购物袋'].index)

In [61]:
data['商品名称'].unique()

array(['古龙沙丁鱼豆豉', '古龙午餐肉', '小英斋鲜肉粽', ..., '三丁包(燕诚)', '鲜肉包(燕诚)', '红豆粽(燕诚)'],
      dtype=object)

# 2部署爬虫

In [1]:
#没有这些库的请自行安装
import re,urllib
from bs4 import BeautifulSoup
import json,time
import requests

In [2]:
def get_search_soup(keywords):
    """
    对京东搜索模块进行解析
    返回搜索商品的网页解析
    html=requests.get(url=url,headers=header,proxies = proxies)
    ps:proxies为代理服务器默认为自己的，针对反爬虫可
    自行去网上找些代理
    """
    #print(type(keyword))
    for i in range(1,2):
        #proxies = {
  #"https": "http://60.7.173.140:4285"
  #"https": "http://10.10.1.10:1080",
      #  }

        url="https://search.jd.com/Search?keyword={}&enc=utf-8&page={}".format(keywords,i*2-1)
        header={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"
        }#伪装成浏览器
        html=requests.get(url=url,headers=header)
        html.encoding="utf-8"
    #print(html.text)
    soup = BeautifulSoup(html.text, 'html.parser')#解析html文件
    return soup

In [63]:
def get_item_urls(soup):
    """
    对搜索后的html进行解析
    返回商品的编号
    """
    list = []#第一页商品的链接
    for i in soup.find_all("div", {"class": "p-img"}):#.find_all("a",{"target":"_blank"}))#.get('href')
        a = i.find("a",{"target":"_blank"})
        href  = a.get('href') #'返回商品的编号：每个京东商品详细网页都由商品编号组成
        list.append(href)
    return list

In [64]:
def get_items_soup(item_urls):
    """
    进入第一个商品的页面，解析商品界面的html
    
    """
    item_url = item_urls[0]#可变参数。第一页第几个商品，得到第一个商品的编号网址
    url_items="https:{}".format(item_url,0)
    header={
    #":authority":"search.jd.com",
    #":method":"GET",
    #":scheme":" https",
        #"upgrade - insecure - requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"
        }
    html=requests.get(url=url_items,headers=header)
    html.encoding="utf-8"
    soup_item = BeautifulSoup(html.text, 'html.parser')#解析html文件
    return soup_item


In [65]:
def get_item_class(items_soup):
    """
    在上面解析的Html中搜寻类别标签，及我们所需要的类别
    """
    class_item = []
    for m in items_soup.find_all("div", {"class": "crumb fl clearfix"}):
        for n in range(1,4):#这里取3级标签
            f = m.find("a",{"clstag":"shangpin|keycount|product|mbNav-"+str(n)})
            name =f.get_text()
            class_item.append(name)
    return class_item

In [66]:
import random
class_all_items = []
for q in data['商品名称'].unique():
    time.sleep(random.randint(1,4))
    keywords = q
    soup = get_search_soup(keywords)
    item_urls = get_item_urls(soup)
    
    if item_urls == []:
        class_items = '没找着'
        class_all_items.append(class_items)
    else:
        items_soup = get_items_soup(item_urls)
        
        item_class = get_item_class(items_soup)
        print(item_class)
        class_all_items.append(get_item_class(items_soup)) 

['食品饮料', '粮油调味', '方便食品']
['食品饮料', '粮油调味', '方便食品']
['食品饮料', '节庆食品/礼券', '粽子']
['生鲜', '速食熟食', '肉制品']
['食品饮料', '粮油调味', '调味品']
['食品饮料', '休闲食品', '饼干蛋糕']
['食品饮料', '休闲食品', '肉干肉脯']
['生鲜', '蔬菜', '茄果瓜类']
['个人护理', '美发假发/造型', '弹力素']
['个人护理', '美发假发/造型', '啫喱膏/水']
['个人护理', '美发假发/造型', '啫喱膏/水']
['个人护理', '洗发护发', '护发素']
['个人护理', '洗发护发', '护发素']
['美妆护肤', '面部护肤', '洁面']
['个人护理', '身体护理', '护手霜']
['美妆护肤', '男士面部护肤', '男士洁面']
['美妆护肤', '面部护肤', '润唇膏']
['个人护理', '身体护理', '润肤']
['美妆护肤', '面部护肤', '润唇膏']
['美妆护肤', '面部护肤', '润唇膏']
['美妆护肤', '面部护肤', '润唇膏']
['家庭清洁/纸品', '衣物清洁', '洗衣液']
['家庭清洁/纸品', '家庭环境清洁', '洗洁精']
['家庭清洁/纸品', '衣物清洁', '洗衣液']
['家庭清洁/纸品', '衣物清洁', '洗衣液']
['家庭清洁/纸品', '家庭环境清洁', '洗洁精']
['家庭清洁/纸品', '衣物清洁', '洗衣粉']
['美妆护肤', '男士面部护肤', '套装/礼盒']
['家庭清洁/纸品', '清洁用具', '垃圾袋/垃圾桶']
['个人护理', '身体护理', '护手霜']
['美妆护肤', '面部护肤', '乳液/面霜']
['美妆护肤', '面部护肤', '乳液/面霜']
['个人护理', '口腔护理', '牙膏']
['个人护理', '口腔护理', '牙膏']
['个人护理', '女性护理', '卫生巾']
['家庭清洁/纸品', '衣物清洁', '洗衣粉']
['家庭清洁/纸品', '衣物清洁', '洗衣皂']
['个人护理', '身体护理', '香皂']
['个人护理', '身体护理', '香皂']
['个人护理', '

['家装建材', '电工电料', '开关插座']
['礼品', '火机烟具', '打火机']
['厨具', '水具酒具', '保温杯']
['厨具', '厨房配件', '保鲜盒']
['厨具', '水具酒具', '保温杯']
['厨具', '水具酒具', '塑料杯']
['厨具', '厨房配件', '保鲜盒']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '进口食品', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '粮油调味', '方便食品']
['食品饮料', '饮料冲调', '饮料']
['食品饮料', '饮料冲

KeyboardInterrupt: 

In [80]:
df_1 = pd.DataFrame(class_all_items)  #写入Pandas进行保存
df_1.to_csv("分类结果.csv",encoding = "utf_8_sig")

## 3拼接模块

就是简单的列表连接，根据自己需求连接就好

设定按"商品名称"当成key，merge一下就行